# Stage 2A: Feature Combination and ML Data Preparation

This notebook implements the first part of Stage 2, focusing on combining molecular features with spectral data and preparing ML-ready datasets.

## Pipeline Overview:

### Responsibilities:
1. **Load Intermediate Data**: Read molecular features and spectral data from Stage 1
2. **Process Spectra**: Apply binning for regression and preserve full resolution for seq2seq
3. **Combine Data**: Merge molecular features with spectral data
4. **Preprocess Features**: Scale, filter, and prepare features for ML
5. **Split Data**: Create train/validation/test sets
6. **Generate ML Formats**:
   - Regression format with binned spectra
   - Seq2seq format with PCA-reduced inputs and full resolution outputs
7. **Save Visualization Data**: Store sample data for Stage 2B visualization

### Input (from Stage 1):
- `data/tmp/{dataset_type}/raw_spectral_data.jsonl`
- `data/tmp/{dataset_type}/molecular_features.jsonl`
- `data/tmp/{dataset_type}/dataset_config.json`

### Output:
#### Regression Format (`data/results/{dataset_type}/full_featurised/`):
- `train_data.jsonl`, `val_data.jsonl`, `test_data.jsonl`
- `feature_preprocessor.pkl` - Scaling and preprocessing state
- `feature_mapping.jsonl` - Feature metadata
- `visualization_data.pkl` - Sample data for Stage 2B

#### Seq2Seq Format (`data/results/{dataset_type}/seq2seq_featurised/`):
- `train_data.jsonl`, `val_data.jsonl`, `test_data.jsonl`
- `pca_transformer.pkl` - PCA transformation matrix
- `seq2seq_config.json` - Configuration metadata

## 1. Environment Setup

Import required libraries for spectral processing and ML preparation.

In [1]:
#!/usr/bin/env python
# coding: utf-8

# Standard library imports
import os
import json
import numpy as np
import pandas as pd
import logging
import traceback
import random
from tqdm import tqdm
from joblib import Parallel, delayed, dump, load
from dataclasses import dataclass, field
from typing import List, Optional
import psutil

# Machine learning imports
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

print("Environment setup complete")
print("=" * 60)

Environment setup complete


## 2. Configuration

Configuration for Stage 2A focusing on data processing and ML preparation.

**Key Configuration Sections:**
- `spectral`: Binning and peak processing parameters
- `seq2seq`: Sequence-to-sequence model settings
- `feature_scaling`: Scaling options for input features (molecular descriptors)
- `target_scaling`: Scaling options for targets (spectra)
- `data_split`: Train/validation/test split ratios

In [2]:
# Stage 2A Configuration
STAGE2A_CONFIG = {
    # Dataset configuration (must match Stage 1)
    'dataset_type': 'hpj',  # Must match the dataset processed in Stage 1
    
    # Paths configuration
    'paths': {
        'data_root': '../data',
        'temp_dir': lambda dtype: f"../data/tmp/{dtype}",  # Input from Stage 1
        'results_dir': lambda dtype: f"../data/results/{dtype}",
        'output_subdir': "full_featurised",
        'output_dir': lambda dtype: f"../data/results/{dtype}/full_featurised",
    },
    
    # Processing configuration
    'processing': {
        'n_jobs': -1,  # Use all available cores
        'save_format': 'jsonl',
        'save_visualization_sample': True,  # Save sample for Stage 2B
        'visualization_sample_size': 100,
    },
    
    # Spectral processing configuration
    'spectral': {
        'max_peaks': 499,
        'bin_size': 1.0,  # For regression only
        'max_mz': 499,
        'sort_peaks_by': 'intensity',
        'intensity_distribution_bins': 20,
        'peak_distribution_bins': 50,
    },
    
    # Seq2seq configuration
    'seq2seq': {
        'enabled': False,  # Toggle to enable/disable seq2seq generation
        'pca_components': 500,  # PCA dimension for input sequence
        'output_subdir': 'seq2seq_featurised',  # Parallel to 'full_featurised'
        'max_sequence_length': 499,  # Same as max_peaks
        'include_regression': True,  # Also keep regression format
        'use_binning': False  # No binning for seq2seq
    },
    
    # Feature scaling configuration (for molecular descriptors/fingerprints)
    'feature_scaling': {
        # Continuous features scaling
        'scale_continuous': False,  # Whether to scale continuous features
        'continuous_scaling_method': 'standard',  # Options: 'standard', 'minmax', 'robust', 'none'
        'apply_log_transform': False,  # Apply sign-preserving log transform before scaling
        
        # Binary features scaling
        'scale_binary': False,  # Whether to scale binary features
        'binary_scaling_method': 'none',  # Options: 'standard', 'none'
        
        # Feature filtering
        'handle_nan_strategy': 'drop_feature',  # Options: 'drop_feature', 'fill_zero', 'fill_mean'
        'min_variance_threshold': 1e-8,
        'auto_detect_binary': True,
    },
    
    # Target scaling configuration (for spectra)
    'target_scaling': {
        'enabled': False,  # Whether to scale target spectra
        'scaling_method': 'minmax',  # Options: 'standard', 'minmax', 'robust', 'none'
        'fit_on': 'train',  # Options: 'train', 'all'
    },
    
    # Data splitting configuration
    'data_split': {
        'random_seed': 41,
        'train_size': 0.8,
        'val_size': 0.1,
        'test_size': 0.1
    }
}

print(f"Configuration loaded for dataset: {STAGE2A_CONFIG['dataset_type']}")
print(f"Seq2seq generation: {'ENABLED' if STAGE2A_CONFIG['seq2seq']['enabled'] else 'DISABLED'}")
print("\nScaling Configuration:")
print(f"  Features:")
print(f"    - Continuous: {STAGE2A_CONFIG['feature_scaling']['scale_continuous']} (method: {STAGE2A_CONFIG['feature_scaling']['continuous_scaling_method']})")
print(f"    - Log transform: {STAGE2A_CONFIG['feature_scaling']['apply_log_transform']}")
print(f"    - Binary: {STAGE2A_CONFIG['feature_scaling']['scale_binary']} (method: {STAGE2A_CONFIG['feature_scaling']['binary_scaling_method']})")
print(f"  Targets:")
print(f"    - Enabled: {STAGE2A_CONFIG['target_scaling']['enabled']} (method: {STAGE2A_CONFIG['target_scaling']['scaling_method']})")
print("=" * 60)

Configuration loaded for dataset: hpj
Seq2seq generation: DISABLED

Scaling Configuration:
  Features:
    - Continuous: False (method: standard)
    - Log transform: False
    - Binary: False (method: none)
  Targets:
    - Enabled: False (method: minmax)


## 3. Validate Stage 1 Output

Check that Stage 1 has been completed and all required files exist.

In [3]:
# Validate Stage 1 completion
dataset_type = STAGE2A_CONFIG['dataset_type']
temp_dir = STAGE2A_CONFIG['paths']['temp_dir'](dataset_type)

# Check required files
required_files = [
    'raw_spectral_data.jsonl',
    'molecular_features.jsonl',
    'dataset_config.json'
]

missing_files = []
for file in required_files:
    file_path = os.path.join(temp_dir, file)
    if not os.path.exists(file_path):
        missing_files.append(file)

if missing_files:
    print(f"ERROR: Stage 1 output incomplete for dataset '{dataset_type}'")
    print(f"Missing files: {missing_files}")
    print(f"Please run Stage 1 (01_molecular_featurization.ipynb) first.")
else:
    # Load dataset config from Stage 1
    with open(os.path.join(temp_dir, 'dataset_config.json'), 'r') as f:
        stage1_config = json.load(f)
    
    print(f"Stage 1 output validated successfully")
    print(f"Dataset: {stage1_config['dataset_type']}")
    print(f"Number of molecules: {stage1_config['num_molecules']}")
    print(f"Stage 1 complete: {stage1_config.get('stage1_complete', False)}")
    print("=" * 60)

Stage 1 output validated successfully
Dataset: hpj
Number of molecules: 2720
Stage 1 complete: True


## 4. Directory Setup

Create output directories for ML-ready datasets.

In [4]:
def setup_directories(dataset_type):
    """Create all necessary directories for Stage 2A."""
    dirs_to_create = [
        STAGE2A_CONFIG['paths']['results_dir'](dataset_type),
        STAGE2A_CONFIG['paths']['output_dir'](dataset_type)
    ]
    
    # Add seq2seq directory if enabled
    if STAGE2A_CONFIG['seq2seq']['enabled']:
        seq2seq_dir = os.path.join(
            STAGE2A_CONFIG['paths']['results_dir'](dataset_type),
            STAGE2A_CONFIG['seq2seq']['output_subdir']
        )
        dirs_to_create.append(seq2seq_dir)
    
    for dir_path in dirs_to_create:
        os.makedirs(dir_path, exist_ok=True)
        logger.info(f"Created directory: {dir_path}")

# Setup directories
print("Setting up output directories...")
setup_directories(STAGE2A_CONFIG['dataset_type'])
print("Directory setup complete")
print("=" * 60)

2025-06-23 09:05:12,160 - INFO - Created directory: ../data/results/hpj
2025-06-23 09:05:12,161 - INFO - Created directory: ../data/results/hpj/full_featurised


Setting up output directories...
Directory setup complete


## 5. Utility Functions and Base Classes

Core utilities needed for Stage 2A processing.

In [5]:
# ---------------------- Utilities ---------------------- #
class Utilities:
    @staticmethod
    def get_memory_usage():
        """Get current memory usage in MB."""
        process = psutil.Process(os.getpid())
        mem_info = process.memory_info()
        return f"{mem_info.rss / (1024 * 1024):.1f} MB (RSS), {process.memory_percent():.1f}% of total"

    @staticmethod
    def ensure_numpy_array(data):
        """Ensure data is a numpy array."""
        if isinstance(data, list):
            return np.array(data)
        return data
    
    @staticmethod
    def convert_np_to_list(item):
        """Recursively convert numpy arrays to lists for JSON serialization."""
        if isinstance(item, np.ndarray):
            return item.tolist()
        elif isinstance(item, dict):
            return {k: Utilities.convert_np_to_list(v) for k, v in item.items()}
        elif isinstance(item, list):
            return [Utilities.convert_np_to_list(v) for v in item]
        else:
            return item
    
    @staticmethod
    def get_scaler(method):
        """Get sklearn scaler based on method name."""
        scalers = {
            'standard': StandardScaler(),
            'minmax': MinMaxScaler(),
            'robust': RobustScaler(),
            'none': None
        }
        return scalers.get(method, StandardScaler())

# ---------------------- Error Handling Mixin ---------------------- #
class ErrorHandlingMixin:
    """Provides standardized error handling for pipeline components."""
    
    def handle_error(self, error, context="", data=None):
        """Centralized error handling."""
        message = f"Error in {self.__class__.__name__}"
        if context:
            message += f" ({context})"
        message += f": {error}"
        
        logger.error(message)
        return None

# ---------------------- Feature Metadata ---------------------- #
@dataclass
class FeatureMetadata:
    feature_names: List[str]
    feature_types: List[str]
    segment_lengths: List[int]
    valid_mask: Optional[np.ndarray] = field(default=None)

    def total_features(self):
        return sum(self.segment_lengths)

print("Utility functions and classes loaded")
print(f"Initial memory usage: {Utilities.get_memory_usage()}")
print("=" * 60)

Utility functions and classes loaded
Initial memory usage: 213.7 MB (RSS), 0.2% of total


## 6. Data Loading Components

Load intermediate data from Stage 1.

In [6]:
# ---------------------- DataLoader ---------------------- #
class DataLoader(ErrorHandlingMixin):
    def __init__(self, config, dataset_type):
        self.config = config
        self.dataset_type = dataset_type
    
    def load_data_from_jsonl(self, file_path):
        """Load data from JSONL file."""
        logger.info(f"Loading data from {file_path}")
        data = {}
        with open(file_path, 'r') as f:
            for line in tqdm(f, desc=f"Loading {os.path.basename(file_path)}"):
                try:
                    record = json.loads(line)
                    smiles = record.get("smiles")
                    data_content = record.get("data", {})
                    data[smiles] = data_content
                except Exception as e:
                    self.handle_error(e, f"loading JSON line from {file_path}")
        logger.info(f"Loaded {len(data)} records from {file_path}")
        return data
    
    def convert_raw_data_to_df(self, raw_data):
        """Convert raw data records to pandas DataFrames."""
        df_data = {}
        for smiles, peaks in raw_data.items():
            df_data[smiles] = pd.DataFrame(peaks, columns=["mz", "intensity"])
        return df_data

print("Data loading components initialized")
print("=" * 60)

Data loading components initialized


## 7. Spectral Processing Components

Process mass spectra with binning for regression and full resolution for seq2seq.

In [7]:
# ---------------------- SpectralProcessor ---------------------- #
class SpectralProcessor(ErrorHandlingMixin):
    def __init__(self, config, dataset_type):
        self.config = config
        self.dataset_type = dataset_type
        self.max_observed_mz = 0
        
    def process(self, raw_spectra, use_binning=True):
        """
        Process spectral data with optional binning.
        
        Args:
            raw_spectra: Dictionary of SMILES -> peak DataFrame
            use_binning: Whether to bin the spectra (True for regression, False for seq2seq)
        """
        logger.info(f"Processing spectral sequences (binning={'enabled' if use_binning else 'disabled'})...")
        
        results = Parallel(n_jobs=self.config['processing']['n_jobs'])(
            delayed(self._process_one_spectrum)(smi, comp, use_binning) 
            for smi, comp in tqdm(list(raw_spectra.items()), desc="Processing spectra")
        )
        
        # Memory-optimized: directly build dictionary
        processed_sequences = {}
        for result in results:
            if result is not None:
                smi, proc = result
                processed_sequences[smi] = proc
        
        # Calculate the maximum observed m/z value across all spectra
        self.max_observed_mz = max([data.get('true_max_mz', 0) for data in processed_sequences.values()], default=0)
        logger.info(f"Maximum observed m/z across all spectra: {self.max_observed_mz}")
        
        if use_binning:
            logger.info(f"Configured max_mz for binning: {self.config['spectral']['max_mz']}")
            
            # Find what percentage of spectra have peaks beyond the max_mz
            spectra_with_peaks_beyond_max = sum(1 for data in processed_sequences.values() 
                                                if data.get('true_max_mz', 0) > self.config['spectral']['max_mz'])
            if processed_sequences:
                percentage_beyond = (spectra_with_peaks_beyond_max / len(processed_sequences)) * 100
                logger.info(f"{spectra_with_peaks_beyond_max} spectra ({percentage_beyond:.2f}%) have peaks beyond configured max_mz")
        
        logger.info(f"Processed spectral sequences for {len(processed_sequences)} compounds")
        return processed_sequences
    
    def _process_one_spectrum(self, smiles, compound_data, use_binning):
        """Process a single spectrum with optional binning."""
        data = compound_data.copy()
        try:
            # Initialize required result variables
            peak_array = np.zeros((self.config['spectral']['max_peaks'], 2), dtype=np.float32)
            attention_mask = np.zeros(self.config['spectral']['max_peaks'], dtype=np.float32)
            original_peak_count = 0
            peak_distribution = np.zeros(self.config['spectral']['intensity_distribution_bins'], dtype=np.float32)
            max_mz_value = 0
            
            # Capture the true maximum m/z value before any filtering
            true_max_mz = data['mz'].max() if not data.empty else 0
            
            # Apply binning only if requested (for regression)
            if use_binning and self.config['spectral']['bin_size'] and not data.empty:
                data['binned_mz'] = (data['mz'] / self.config['spectral']['bin_size']).astype(int) * self.config['spectral']['bin_size']
                data = data.groupby('binned_mz').agg({'intensity': 'max'}).reset_index().rename(columns={'binned_mz': 'mz'})
            
            # Count peaks beyond max_mz (only relevant for binned data)
            peaks_beyond_max = 0
            if use_binning:
                peaks_beyond_max = sum(1 for mz in data['mz'] if mz > self.config['spectral']['max_mz']) if not data.empty else 0
            
            # Process peaks if data is not empty
            if not data.empty:
                sorted_peaks = data.sort_values(self.config['spectral']['sort_peaks_by'], ascending=False)
                max_intensity = sorted_peaks['intensity'].max() if not sorted_peaks.empty else 0
                if max_intensity > 0:
                    sorted_peaks['intensity'] /= max_intensity
                
                peak_sequence = list(zip(sorted_peaks['mz'], sorted_peaks['intensity']))
                original_peak_count = len(peak_sequence)
                
                if len(peak_sequence) > 0:
                    # Process peak sequences
                    if len(peak_sequence) > self.config['spectral']['max_peaks']:
                        attention_mask = np.ones(self.config['spectral']['max_peaks'])
                        peak_sequence = peak_sequence[:self.config['spectral']['max_peaks']]
                    else:
                        attention_mask = np.concatenate([
                            np.ones(len(peak_sequence)), 
                            np.zeros(self.config['spectral']['max_peaks'] - len(peak_sequence))
                        ])
                        peak_sequence += [(0.0, 0.0)] * (self.config['spectral']['max_peaks'] - len(peak_sequence))
                    
                    peak_array = np.array(peak_sequence, dtype=np.float32)
                    peak_distribution = self._create_intensity_distribution(
                        peak_array, attention_mask, self.config['spectral']['intensity_distribution_bins']
                    )
                    
                    max_mz_value = sorted_peaks['mz'].max() if not sorted_peaks.empty else 0
            
            # Free up memory
            del data
            
            result = {
                'peaks': peak_array,
                'attention_mask': attention_mask,
                'original_peak_count': original_peak_count,
                'intensity_distribution': peak_distribution,
                'max_mz': max_mz_value,
                'true_max_mz': true_max_mz,
                'binned': use_binning
            }
            
            if use_binning:
                result.update({
                    'peaks_beyond_max': peaks_beyond_max,
                    'bin_size': self.config['spectral']['bin_size']
                })
            
            return smiles, result
            
        except Exception as e:
            return self.handle_error(e, f"processing spectrum for {smiles}", data={"smiles": smiles})
    
    def _create_intensity_distribution(self, peak_array, mask, num_bins):
        """Calculate intensity distribution histogram."""
        valid_peaks = peak_array[mask.astype(bool)]
        if len(valid_peaks) == 0:
            return np.zeros(num_bins)
        hist, _ = np.histogram(valid_peaks[:, 1], bins=num_bins, range=(0, 1))
        return hist / np.sum(hist) if np.sum(hist) > 0 else hist

print("Spectral processing components loaded")
print("=" * 60)

Spectral processing components loaded


## 8. Data Combination and Binning Components

In [8]:
# ---------------------- DataCombiner ---------------------- #
class DataCombiner(ErrorHandlingMixin):
    def __init__(self, config, dataset_type):
        self.config = config
        self.dataset_type = dataset_type
        
    def process(self, spectra_sequences, molecular_features):
        """Combine spectral and molecular data."""
        combined_data = {}
        missing_features = 0
        
        for smiles, spectral_data in tqdm(spectra_sequences.items(), desc="Combining data"):
            if smiles in molecular_features:
                combined_data[smiles] = self._combine_single_molecule(smiles, spectral_data, molecular_features[smiles])
            else:
                missing_features += 1
                
        logger.info(f"Combined data: {len(combined_data)} compounds, missing features for {missing_features}")
        return combined_data
    
    def _combine_single_molecule(self, smiles, spectral_data, features):
        """Combine data for a single molecule."""
        molecular_weight = exact_mass = 0
        
        if 'descriptors' in features and 'descriptor_names' in features:
            try:
                molwt_idx = features['descriptor_names'].index('MolWt')
                molecular_weight = float(features['descriptors'][molwt_idx])
            except (ValueError, IndexError):
                pass
            try:
                exactwt_idx = features['descriptor_names'].index('ExactMolWt')
                exact_mass = float(features['descriptors'][exactwt_idx])
            except (ValueError, IndexError):
                pass
                
        # Combine all data
        combined = {}
        # First add spectral data
        for key, value in spectral_data.items():
            combined[key] = value
        # Then add features
        for key, value in features.items():
            combined[key] = value
        combined['molecular_weight'] = molecular_weight
        combined['exact_mass'] = exact_mass
        
        return combined

# ---------------------- BinningProcessor ---------------------- #
class BinningProcessor(ErrorHandlingMixin):
    def __init__(self, config, dataset_type):
        self.config = config
        self.dataset_type = dataset_type
        self.max_observed_mz = 0
        
    def process(self, combined_data):
        """Create binned representations for regression."""
        # First find the maximum observed m/z value across all spectra
        self.max_observed_mz = max([data.get('true_max_mz', 0) for data in combined_data.values()], default=0)
        logger.info(f"Maximum observed m/z for binning: {self.max_observed_mz}")
        
        results = Parallel(n_jobs=self.config['processing']['n_jobs'])(
            delayed(self._process_binning)(smi, data) 
            for smi, data in tqdm(list(combined_data.items()), desc="Binning spectra")
        )
        
        # Memory-optimized dictionary construction
        binned_data = {}
        for result in results:
            if result is not None:
                smi, proc = result
                binned_data[smi] = proc
                
        logger.info(f"Created binned representations for {len(binned_data)} compounds")
        logger.info(f"Memory usage after binning: {Utilities.get_memory_usage()}")
        
        return binned_data
    
    def _process_binning(self, smiles, data):
        """Process binning for a single molecule."""
        peaks = Utilities.ensure_numpy_array(data['peaks'])
        mask = Utilities.ensure_numpy_array(data['attention_mask'])
        real_peaks = peaks[mask.astype(bool)]
        
        # Create binned spectrum up to configured max_mz
        configured_num_bins = int(self.config['spectral']['max_mz'] / self.config['spectral']['bin_size']) + 1
        binned_spectrum = np.zeros(configured_num_bins)
        
        # Create extended binned spectrum up to observed max_mz (for visualization)
        extended_num_bins = int(max(self.max_observed_mz * 1.1, self.config['spectral']['max_mz']) / 
                              self.config['spectral']['bin_size']) + 1
        extended_binned_spectrum = np.zeros(extended_num_bins)
        
        for mz, intensity in real_peaks:
            # For the standard binned spectrum (used in the pipeline)
            if mz < self.config['spectral']['max_mz']:
                bin_idx = min(int(mz / self.config['spectral']['bin_size']), configured_num_bins - 1)
                binned_spectrum[bin_idx] = max(binned_spectrum[bin_idx], intensity)
            
            # For the extended binned spectrum (used in visualization)
            extended_bin_idx = min(int(mz / self.config['spectral']['bin_size']), extended_num_bins - 1)
            extended_binned_spectrum[extended_bin_idx] = max(extended_binned_spectrum[extended_bin_idx], intensity)
        
        # Count peaks and intensity beyond max_mz
        peaks_beyond_max = [(mz, intensity) for mz, intensity in real_peaks if mz > self.config['spectral']['max_mz']]
        total_intensity_beyond_max = sum(intensity for _, intensity in peaks_beyond_max)
        total_intensity = sum(intensity for _, intensity in real_peaks)
        intensity_percentage_beyond_max = (total_intensity_beyond_max / total_intensity * 100) if total_intensity > 0 else 0
        
        # Create a new data dictionary
        updated = {}
        # Copy original data that's needed
        for key in ['peaks', 'attention_mask', 'original_peak_count', 
                   'intensity_distribution', 'max_mz', 'true_max_mz', 
                   'peaks_beyond_max', 'bin_size', 
                   'molecular_weight', 'exact_mass']:
            if key in data:
                updated[key] = data[key]
                
        # Add new binning data
        updated['binned_spectrum'] = binned_spectrum
        updated['extended_binned_spectrum'] = extended_binned_spectrum
        updated['peaks_beyond_max_count'] = len(peaks_beyond_max)
        updated['intensity_beyond_max'] = total_intensity_beyond_max
        updated['intensity_percentage_beyond_max'] = intensity_percentage_beyond_max
        updated['extended_max_mz'] = extended_num_bins * self.config['spectral']['bin_size']
        
        # Keep molecular feature data
        for key in data:
            if key not in updated and (key.endswith('fingerprint') or key.endswith('descriptors') or 
                                      key.endswith('feature_names') or key.endswith('features')):
                updated[key] = data[key]
        
        return smiles, updated

print("Data combination and binning components loaded")
print("=" * 60)

Data combination and binning components loaded


## 9. Data Preparation Components

In [9]:
# ---------------------- RegressionDataPreparer ---------------------- #
class RegressionDataPreparer(ErrorHandlingMixin):
    def __init__(self, config, dataset_type):
        self.config = config
        self.dataset_type = dataset_type
        self.target_scaler = None
    
    def process(self, data_dict):
        """Extract features and targets for regression."""
        logger.info("Extracting features and targets for regression...")
        
        first_key = next(iter(data_dict.keys()), None)
        feature_metadata = None
        if first_key:
            feature_metadata = self._extract_feature_info(data_dict[first_key])
        
        results = []
        for smi, data in tqdm(list(data_dict.items()), desc="Preparing regression data"):
            result = self._process_regression_sample(smi, data)
            if result is not None:
                results.append(result)
        
        if not results:
            logger.error("No valid regression samples found!")
            return None
        
        # Unpack results in a memory-efficient way
        smiles_list = []
        feature_list = []
        target_list = []
        for result in results:
            smiles, features, target = result
            smiles_list.append(smiles)
            feature_list.append(features)
            target_list.append(target)
        
        # Convert to numpy arrays
        X = np.array(feature_list)
        y = np.array(target_list)
        
        # Clear original results list to free memory
        del results
        del feature_list
        del target_list
        
        logger.info(f"Memory usage after regression data preparation: {Utilities.get_memory_usage()}")
        
        if feature_metadata:
            expected_count = feature_metadata.total_features()
            actual_count = X.shape[1]
            if expected_count != actual_count:
                logger.warning(f"Feature dimension mismatch: expected {expected_count}, got {actual_count}")
                feature_metadata.feature_names = [f"feature_{i}" for i in range(actual_count)]
                feature_metadata.feature_types = ["unknown"] * actual_count
                feature_metadata.segment_lengths = [actual_count]
        
        logger.info(f"Prepared regression matrices - X: {X.shape}, y: {y.shape}")
        return X, y, smiles_list, feature_metadata
    
    def scale_targets(self, y_train, y_val=None, y_test=None):
        """Scale target spectra if enabled."""
        if not self.config['target_scaling']['enabled']:
            logger.info("Target scaling disabled, returning original targets")
            return y_train, y_val, y_test
        
        logger.info(f"Scaling targets using method: {self.config['target_scaling']['scaling_method']}")
        
        # Get the appropriate scaler
        self.target_scaler = Utilities.get_scaler(self.config['target_scaling']['scaling_method'])
        
        if self.target_scaler is None:
            return y_train, y_val, y_test
        
        # Fit scaler on training data
        y_train_scaled = self.target_scaler.fit_transform(y_train)
        
        # Transform validation and test if provided
        y_val_scaled = self.target_scaler.transform(y_val) if y_val is not None else None
        y_test_scaled = self.target_scaler.transform(y_test) if y_test is not None else None
        
        return y_train_scaled, y_val_scaled, y_test_scaled
    
    def _extract_feature_info(self, data) -> FeatureMetadata:
        """Extract feature metadata."""
        names = []
        types = []
        segments = []
        
        if 'descriptors' in data and data['descriptors'] is not None and 'descriptor_names' in data:
            names.extend(data['descriptor_names'])
            types.extend(['descriptor'] * len(data['descriptor_names']))
            segments.append(len(data['descriptors']))
            
        if 'morgan_fingerprint' in data and data['morgan_fingerprint'] is not None:
            names.extend([f'morgan_bit_{i}' for i in range(len(data['morgan_fingerprint']))])
            types.extend(['morgan_fingerprint'] * len(data['morgan_fingerprint']))
            segments.append(len(data['morgan_fingerprint']))
        
        additional_keys = ['morgan_feature_fp', 'atom_pairs', 'electronic_features', 'substructure_features',
                           'rdkit_fingerprint', 'avalon_fingerprint', 'pattern_fingerprint', 'layered_fingerprint']
        for key in additional_keys:
            if key in data and data[key] is not None:
                if f'{key}_names' in data:
                    names.extend(data[f'{key}_names'])
                    types.extend([key] * len(data[f'{key}_names']))
                else:
                    names.extend([f'{key}_{i}' for i in range(len(data[key]))])
                    types.extend([key] * len(data[key]))
                segments.append(len(data[key]))
                
        return FeatureMetadata(feature_names=names, feature_types=types, segment_lengths=segments)
    
    def _process_regression_sample(self, smiles, data):
        """Process a single regression sample."""
        features = []
        
        # Use a more memory-efficient approach for data conversion
        feature_data = {}
        for k, v in data.items():
            if isinstance(v, (list, np.ndarray)):
                feature_data[k] = Utilities.ensure_numpy_array(v)
        
        if 'descriptors' in feature_data and feature_data['descriptors'] is not None:
            features.append(feature_data['descriptors'])
        if 'morgan_fingerprint' in feature_data and feature_data['morgan_fingerprint'] is not None:
            features.append(feature_data['morgan_fingerprint'])
        
        additional_keys = ['morgan_feature_fp', 'atom_pairs', 'electronic_features', 'substructure_features',
                           'rdkit_fingerprint', 'avalon_fingerprint', 'pattern_fingerprint', 'layered_fingerprint']
        for key in additional_keys:
            if key in feature_data and feature_data[key] is not None:
                features.append(feature_data[key])
        
        if not features or 'binned_spectrum' not in feature_data:
            return None
        
        try:
            concatenated = np.concatenate(features)
            return (smiles, concatenated, feature_data['binned_spectrum'])
        except Exception as e:
            return self.handle_error(e, f"concatenating features for {smiles}", data={"smiles": smiles})

# ---------------------- Seq2SeqDataPreparer ---------------------- #
class Seq2SeqDataPreparer(ErrorHandlingMixin):
    def __init__(self, config, dataset_type):
        self.config = config
        self.dataset_type = dataset_type
        self.pca = None
        
    def process(self, X_scaled, combined_data, smiles_list):
        """
        Convert regression features to seq2seq format.
        
        Args:
            X_scaled: Scaled feature matrix from regression preprocessing
            combined_data: Dictionary with full resolution spectral data
            smiles_list: List of SMILES strings in same order as X_scaled
            
        Returns:
            List of seq2seq data dictionaries
        """
        logger.info("Preparing seq2seq data...")
        
        # Apply PCA to reduce features
        logger.info(f"Applying PCA to reduce features from {X_scaled.shape[1]} to {self.config['seq2seq']['pca_components']} dimensions")
        self.pca = PCA(n_components=self.config['seq2seq']['pca_components'])
        X_pca = self.pca.fit_transform(X_scaled)
        
        # Log explained variance
        explained_var = np.sum(self.pca.explained_variance_ratio_)
        logger.info(f"PCA explained variance: {explained_var:.2%}")
        
        # Extract peak sequences from combined_data
        seq2seq_data = []
        
        for i, smiles in enumerate(tqdm(smiles_list, desc="Creating seq2seq samples")):
            if smiles in combined_data:
                data = combined_data[smiles]
                
                # Get full resolution peaks (not binned)
                peaks = data['peaks']  # Already (mz, intensity) pairs
                mask = data['attention_mask']
                
                seq2seq_data.append({
                    'smiles': smiles,
                    'input_sequence': X_pca[i],  # 500D PCA features
                    'output_sequence': peaks,     # Full resolution (mz, intensity) pairs
                    'output_mask': mask,          # Attention mask
                    'original_peak_count': data['original_peak_count'],
                    'molecular_weight': data.get('molecular_weight', 0),
                    'exact_mass': data.get('exact_mass', 0)
                })
        
        logger.info(f"Created {len(seq2seq_data)} seq2seq samples")
        return seq2seq_data
    
    def save_pca(self, filepath):
        """Save PCA transformer for inference."""
        if self.pca is not None:
            dump(self.pca, filepath)
            logger.info(f"Saved PCA transformer to {filepath}")
        else:
            logger.warning("No PCA transformer to save")

print("Data preparation components loaded")
print("=" * 60)

Data preparation components loaded


## 10. Feature Preprocessing

In [10]:
# ---------------------- FeaturePreprocessor ---------------------- #
class FeaturePreprocessor(ErrorHandlingMixin):
    def __init__(self, config, dataset_type):
        self.config = config
        self.dataset_type = dataset_type
        self.binary_feature_mask = None
        self.valid_feature_mask = None
        self.scaler = None
        self.binary_scaler = None
        self.nan_mask = None
    
    def _sign_preserving_log_transform(self, X):
        """Apply sign-preserving log transform: sign(x)*log1p(|x|)."""
        return np.sign(X) * np.log1p(np.abs(X))
    
    def process(self, X, fit=True, metadata: FeatureMetadata = None):
        """Preprocess features with scaling and filtering."""
        if X is None or len(X) == 0:
            logger.warning("Empty feature matrix, skipping preprocessing")
            return None
            
        X = np.array(X) if not isinstance(X, np.ndarray) else X
        
        if fit:
            return self._fit_transform(X, metadata)
        else:
            return self._transform(X, metadata)
    
    def _fit_transform(self, X, metadata: FeatureMetadata = None):
        """Fit preprocessor and transform features."""
        self._handle_nans(X)
        
        if self.config['feature_scaling'].get('auto_detect_binary', True):
            self._detect_binary_features(X)
            
        # Apply the sign-preserving log transform only to continuous features if enabled
        X = X.copy()
        if self.config['feature_scaling'].get('apply_log_transform', True) and self.binary_feature_mask is not None:
            continuous_indices = np.where(~self.binary_feature_mask)[0]
            if len(continuous_indices) > 0:
                X[:, continuous_indices] = self._sign_preserving_log_transform(X[:, continuous_indices])
                logger.info(f"Applied log transform to {len(continuous_indices)} continuous features")
                
        self._create_valid_feature_mask(X, metadata)
        X_valid = X[:, self.valid_feature_mask]
        X_scaled = self._scale_features(X_valid, fit=True)
        
        return X_scaled
    
    def _transform(self, X, metadata: FeatureMetadata = None):
        """Transform features using fitted preprocessor."""
        if self.valid_feature_mask is None:
            logger.error("FeaturePreprocessor not fitted before transform")
            return X
            
        X = self._handle_nan_values(X)
        
        # Apply the sign-preserving log transform to continuous features if enabled
        if self.config['feature_scaling'].get('apply_log_transform', True) and self.binary_feature_mask is not None:
            continuous_indices = np.where(~self.binary_feature_mask)[0]
            if len(continuous_indices) > 0:
                X[:, continuous_indices] = self._sign_preserving_log_transform(X[:, continuous_indices])
                
        X_valid = X[:, self.valid_feature_mask]
        X_scaled = self._scale_features(X_valid, fit=False)
        
        return X_scaled
    
    def _handle_nans(self, X):
        """Check for and handle NaN and infinity values."""
        # Check for NaN values
        nan_counts = np.isnan(X).sum(axis=0)
        # Check for infinity values
        inf_counts = np.isinf(X).sum(axis=0)
        
        total_invalid = nan_counts + inf_counts
        if total_invalid.sum() > 0:
            logger.info(f"Found {nan_counts.sum()} NaN values and {inf_counts.sum()} infinity values across {(total_invalid > 0).sum()} features")
        
        # Create mask for valid values (no NaN or inf)
        self.nan_mask = total_invalid == 0
        
        # Handle invalid values
        X = self._handle_nan_values(X)
        return X
    
    def _handle_nan_values(self, X):
        """Handle both NaN and infinity values based on strategy."""
        strategy = self.config['feature_scaling'].get('handle_nan_strategy', 'drop_feature')
        X_copy = X.copy()
        
        if strategy == 'fill_zero':
            # Handle both NaN and infinity
            X_copy = np.nan_to_num(X_copy, nan=0.0, posinf=0.0, neginf=0.0)
        elif strategy == 'fill_mean':
            # Calculate means ignoring both NaN and infinity
            finite_mask = np.isfinite(X_copy)
            col_means = np.zeros(X_copy.shape[1])
            
            for col_idx in range(X_copy.shape[1]):
                col_data = X_copy[:, col_idx]
                finite_col_data = col_data[finite_mask[:, col_idx]]
                if len(finite_col_data) > 0:
                    col_means[col_idx] = np.mean(finite_col_data)
            
            # Replace both NaN and infinity with means
            non_finite_mask = ~np.isfinite(X_copy)
            for col_idx in range(X_copy.shape[1]):
                X_copy[non_finite_mask[:, col_idx], col_idx] = col_means[col_idx]
        
        return X_copy
    
    def _detect_binary_features(self, X):
        """Detect binary features in the data."""
        unique_vals = [np.unique(X[:, i][np.isfinite(X[:, i])]) for i in range(X.shape[1])]
        is_binary = [len(u) <= 2 and set(u).issubset({0, 1}) for u in unique_vals]
        self.binary_feature_mask = np.array(is_binary)
        binary_count = np.sum(self.binary_feature_mask)
        if binary_count > 0:
            logger.info(f"Detected {binary_count} binary features")
    
    def _create_valid_feature_mask(self, X, metadata: FeatureMetadata = None):
        """Create mask for valid features."""
        valid_mask = np.ones(X.shape[1], dtype=bool)
        strategy = self.config['feature_scaling'].get('handle_nan_strategy', 'drop_feature')
        
        if strategy == 'drop_feature' and self.nan_mask is not None:
            valid_mask = valid_mask & self.nan_mask
            nan_count = np.sum(np.isnan(X).any(axis=0))
            inf_count = np.sum(np.isinf(X).any(axis=0))
            logger.info(f"Dropped {np.sum(~self.nan_mask)} features with invalid values ({nan_count} with NaN, {inf_count} with infinity)")
        
        variance_threshold = self.config['feature_scaling'].get('min_variance_threshold', 1e-8)
        if variance_threshold > 0:
            # Replace infinity with NaN so nanvar can handle it
            X_for_var = X.copy()
            X_for_var[np.isinf(X_for_var)] = np.nan
            variances = np.nanvar(X_for_var, axis=0)
            variances = np.nan_to_num(variances, nan=0.0)  # Replace any NaN variance with 0
            
            high_variance_mask = variances > variance_threshold
            valid_mask = valid_mask & high_variance_mask
            logger.info(f"Dropped {np.sum(~high_variance_mask)} features with variance below {variance_threshold}")
            
            # Free memory
            del X_for_var
            del variances
        
        self.valid_feature_mask = valid_mask
        if metadata is not None:
            metadata.valid_mask = valid_mask
        
        logger.info(f"Final feature count: {np.sum(valid_mask)} of {X.shape[1]} original features")
        return valid_mask
    
    def _scale_features(self, X, fit=True):
        """Scale features with appropriate strategies."""
        if X is None or X.shape[0] == 0 or X.shape[1] == 0:
            return X
            
        scale_continuous = self.config['feature_scaling'].get('scale_continuous', True)
        scale_binary = self.config['feature_scaling'].get('scale_binary', False)
        continuous_method = self.config['feature_scaling'].get('continuous_scaling_method', 'standard')
        binary_method = self.config['feature_scaling'].get('binary_scaling_method', 'none')
        
        # If no scaling is required, clip and return.
        if not scale_continuous and not scale_binary:
            X_clipped = np.clip(X, np.finfo(np.float32).min, np.finfo(np.float32).max)
            return X_clipped.astype(np.float32)
        
        # If binary feature detection is off or not available.
        if self.binary_feature_mask is None or not self.config['feature_scaling'].get('auto_detect_binary', True):
            if scale_continuous:
                if fit:
                    self.scaler = Utilities.get_scaler(continuous_method)
                    if self.scaler is not None:
                        X_scaled = self.scaler.fit_transform(X)
                    else:
                        X_scaled = X
                else:
                    if self.scaler is None:
                        logger.warning("Scaler not fitted before transform. Returning unscaled features.")
                        X_scaled = X
                    else:
                        X_scaled = self.scaler.transform(X)
            else:
                X_scaled = X
            X_clipped = np.clip(X_scaled, np.finfo(np.float32).min, np.finfo(np.float32).max)
            return X_clipped.astype(np.float32)
        
        # When binary feature mask is available.
        binary_valid = self.binary_feature_mask[self.valid_feature_mask]
        X_processed = X.copy()
        
        if scale_continuous:
            continuous_mask = ~binary_valid
            if np.any(continuous_mask):
                continuous_indices = np.where(continuous_mask)[0]
                continuous_data = X[:, continuous_indices]
                if fit:
                    self.scaler = Utilities.get_scaler(continuous_method)
                    if self.scaler is not None:
                        X_processed[:, continuous_indices] = self.scaler.fit_transform(continuous_data)
                        logger.info(f"Applied {continuous_method} scaling to {len(continuous_indices)} continuous features")
                else:
                    if self.scaler is None:
                        logger.warning("Scaler not fitted before transform. Returning unscaled continuous features.")
                    else:
                        X_processed[:, continuous_indices] = self.scaler.transform(continuous_data)
                del continuous_data
                
        if scale_binary and np.any(binary_valid) and binary_method != 'none':
            binary_indices = np.where(binary_valid)[0]
            if fit:
                self.binary_scaler = Utilities.get_scaler(binary_method)
                if self.binary_scaler is not None:
                    X_processed[:, binary_indices] = self.binary_scaler.fit_transform(X[:, binary_indices])
                    logger.info(f"Applied {binary_method} scaling to {len(binary_indices)} binary features")
            else:
                if hasattr(self, 'binary_scaler') and self.binary_scaler is not None:
                    X_processed[:, binary_indices] = self.binary_scaler.transform(X[:, binary_indices])
                    
        X_clipped = np.clip(X_processed, np.finfo(np.float32).min, np.finfo(np.float32).max)
        return X_clipped.astype(np.float32)
    
    def update_metadata(self, metadata: FeatureMetadata):
        """Update the FeatureMetadata by filtering out features dropped by the valid mask."""
        if metadata is None or metadata.valid_mask is None:
            logger.warning("No valid metadata or valid_mask provided for update.")
            return metadata
            
        filtered_names = [name for name, valid in zip(metadata.feature_names, metadata.valid_mask) if valid]
        filtered_types = [typ for typ, valid in zip(metadata.feature_types, metadata.valid_mask) if valid]
        updated_segment_lengths = [len(filtered_names)]
        
        logger.info(f"Updated metadata: {len(filtered_names)} valid features retained out of {metadata.total_features()}.")
        
        return FeatureMetadata(
            feature_names=filtered_names,
            feature_types=filtered_types,
            segment_lengths=updated_segment_lengths,
            valid_mask=np.array([True]*len(filtered_names))
        )
    
    def save(self, output_path):
        """Save preprocessor state."""
        state = {
            'binary_feature_mask': self.binary_feature_mask,
            'valid_feature_mask': self.valid_feature_mask,
            'scaler': self.scaler,
            'binary_scaler': getattr(self, 'binary_scaler', None),
            'nan_mask': getattr(self, 'nan_mask', None),
            'config_feature_scaling': self.config['feature_scaling'],
            'config_spectral': self.config['spectral']
        }
        dump(state, output_path)
        logger.info(f"Saved feature preprocessor to {output_path}")

print("Feature preprocessor loaded")
print("=" * 60)

Feature preprocessor loaded


## 11. Data Saving Components

In [11]:
# ---------------------- FeatureMapper ---------------------- #
class FeatureMapper(ErrorHandlingMixin):
    def __init__(self, config, dataset_type):
        self.config = config
        self.dataset_type = dataset_type
        self.feature_map = {}
        self.output_file = os.path.join(config['paths']['output_dir'](dataset_type), 'feature_mapping.jsonl')
        self.full_mapping_file = os.path.join(config['paths']['output_dir'](dataset_type), 'feature_mapping_full.json')
        
    def collect_feature_info(self, metadata: FeatureMetadata, target_scaler=None):
        """Collect feature mapping information."""
        if metadata.valid_mask is None or len(metadata.valid_mask) != metadata.total_features():
            logger.error("Mismatch between valid mask and total features in metadata.")
            raise ValueError("Inconsistent feature metadata.")
        
        # Store the mapping with original indices (BEFORE filtering)
        feature_indices = [i for i, _ in enumerate(metadata.feature_names)]
        filtered_indices = [i for i, valid in zip(feature_indices, metadata.valid_mask) if valid]
        filtered_names = [name for name, valid in zip(metadata.feature_names, metadata.valid_mask) if valid]
        filtered_types = [typ for typ, valid in zip(metadata.feature_types, metadata.valid_mask) if valid]
        
        # Create feature map for filtered features with original indices
        self.feature_map = {
            i: {
                'name': name, 
                'type': typ, 
                'new_index': i,
                'original_index': original_idx
            }
            for i, (name, typ, original_idx) in enumerate(zip(filtered_names, filtered_types, filtered_indices))
        }
        
        # Save complete feature information for backend
        self.complete_feature_info = {
            'all_feature_names': metadata.feature_names,
            'all_feature_types': metadata.feature_types,
            'valid_mask': metadata.valid_mask.tolist() if isinstance(metadata.valid_mask, np.ndarray) else metadata.valid_mask,
            'filtered_indices': filtered_indices,
            'filtered_names': filtered_names,
            'filtered_types': filtered_types,
            'input_dimension': len(metadata.feature_names),
            'output_dimension': len(filtered_names),
            'feature_scaling_config': self.config['feature_scaling'],
            'target_scaling_config': self.config['target_scaling'],
            'target_scaler_fitted': target_scaler is not None
        }
        
    def save_mapping(self):
        """Save feature mapping to files."""
        if not self.feature_map:
            logger.warning("No feature mapping to save")
            return
            
        # Save individual feature mappings as JSONL (for legacy compatibility)
        os.makedirs(os.path.dirname(self.output_file), exist_ok=True)
        with open(self.output_file, 'w') as f:
            for idx, info in self.feature_map.items():
                mapping = {
                    'index': idx,
                    'name': info['name'],
                    'type': info['type'],
                    'original_index': info['original_index']
                }
                f.write(json.dumps(mapping) + "\n")
                
        # Save complete mapping information as single JSON for easier loading in backend
        with open(self.full_mapping_file, 'w') as f:
            json.dump(self.complete_feature_info, f, indent=2)
                
        logger.info(f"Saved feature mapping for {len(self.feature_map)} features to {self.output_file}")
        logger.info(f"Saved complete feature info to {self.full_mapping_file}")

# ---------------------- DataSaver ---------------------- #
class DataSaver(ErrorHandlingMixin):
    def __init__(self, config, dataset_type):
        self.config = config
        self.dataset_type = dataset_type
        
    def save_split(self, smiles_list, X, y, indices, filepath, is_features_scaled=True, is_targets_scaled=False):
        """Save regression data split."""
        try:
            os.makedirs(os.path.dirname(filepath), exist_ok=True)
            with open(filepath, 'w') as f:
                for idx in tqdm(indices, desc=f"Saving to {os.path.basename(filepath)}"):
                    sample = {
                        "smiles": smiles_list[idx],
                        "features": X[idx].tolist(),
                        "target": y[idx].tolist(),
                        "is_features_scaled": is_features_scaled,
                        "is_targets_scaled": is_targets_scaled
                    }
                    f.write(json.dumps(sample) + "\n")
                    # Clear sample to free memory
                    sample = None
            logger.info(f"Saved {len(indices)} samples to {filepath}")
            return len(indices)
        except Exception as e:
            self.handle_error(e, f"saving data to {filepath}", data={"file_path": filepath, "indices_count": len(indices)})
            return 0
    
    def save_seq2seq_split(self, seq2seq_data, indices, filepath):
        """Save seq2seq formatted data."""
        try:
            os.makedirs(os.path.dirname(filepath), exist_ok=True)
            with open(filepath, 'w') as f:
                for idx in tqdm(indices, desc=f"Saving seq2seq to {os.path.basename(filepath)}"):
                    sample = seq2seq_data[idx]
                    # Convert numpy arrays to lists for JSON
                    json_sample = {
                        "smiles": sample['smiles'],
                        "input_sequence": sample['input_sequence'].tolist(),
                        "output_sequence": sample['output_sequence'].tolist(),
                        "output_mask": sample['output_mask'].tolist(),
                        "original_peak_count": sample['original_peak_count'],
                        "molecular_weight": sample.get('molecular_weight', 0),
                        "exact_mass": sample.get('exact_mass', 0)
                    }
                    f.write(json.dumps(json_sample) + "\n")
            logger.info(f"Saved {len(indices)} seq2seq samples to {filepath}")
        except Exception as e:
            self.handle_error(e, f"saving seq2seq data to {filepath}")
    
    def save_target_scaler(self, scaler, filepath):
        """Save target scaler if target scaling is enabled."""
        if scaler is not None:
            dump(scaler, filepath)
            logger.info(f"Saved target scaler to {filepath}")

print("Data saving components loaded")
print("=" * 60)

Data saving components loaded


## 12. Execute Stage 2A: Process and Prepare ML Data

Run the complete Stage 2A pipeline.

In [12]:
print("STAGE 2A: FEATURE COMBINATION AND ML PREPARATION")
print("=" * 60)

# Load data from Stage 1
print("Loading intermediate data from Stage 1...")

# Define paths
dataset_type = STAGE2A_CONFIG['dataset_type']
temp_dir = STAGE2A_CONFIG['paths']['temp_dir'](dataset_type)
raw_data_path = os.path.join(temp_dir, 'raw_spectral_data.jsonl')
mol_features_path = os.path.join(temp_dir, 'molecular_features.jsonl')

# Initialize data loader
data_loader = DataLoader(STAGE2A_CONFIG, dataset_type)

# Log initial memory usage
print(f"Initial memory usage: {Utilities.get_memory_usage()}")
print("")

# Step 1: Load raw spectral data and convert to DataFrames
print("Loading raw spectral data...")
raw_data_records = data_loader.load_data_from_jsonl(raw_data_path)
raw_data = data_loader.convert_raw_data_to_df(raw_data_records)

# Free up memory
del raw_data_records

print(f"Memory usage after loading raw data: {Utilities.get_memory_usage()}")

# Step 2: Load molecular features
print("\nLoading molecular features...")
molecular_features = data_loader.load_data_from_jsonl(mol_features_path)

print(f"Memory usage after loading molecular features: {Utilities.get_memory_usage()}")
print(f"\nLoaded {len(raw_data)} spectral records and {len(molecular_features)} molecule features")
print("=" * 60)

2025-06-23 09:05:12,465 - INFO - Loading data from ../data/tmp/hpj/raw_spectral_data.jsonl


STAGE 2A: FEATURE COMBINATION AND ML PREPARATION
Loading intermediate data from Stage 1...
Initial memory usage: 217.2 MB (RSS), 0.2% of total

Loading raw spectral data...


Loading raw_spectral_data.jsonl: 2720it [00:00, 5052.98it/s]
2025-06-23 09:05:13,021 - INFO - Loaded 2720 records from ../data/tmp/hpj/raw_spectral_data.jsonl
2025-06-23 09:05:13,395 - INFO - Loading data from ../data/tmp/hpj/molecular_features.jsonl


Memory usage after loading raw data: 307.2 MB (RSS), 0.2% of total

Loading molecular features...


Loading molecular_features.jsonl: 2720it [00:04, 627.68it/s]
2025-06-23 09:05:17,731 - INFO - Loaded 2720 records from ../data/tmp/hpj/molecular_features.jsonl


Memory usage after loading molecular features: 609.6 MB (RSS), 0.5% of total

Loaded 2720 spectral records and 2720 molecule features


In [13]:
# Initialize all processing components
print("Initializing processing components...")
spectral_processor = SpectralProcessor(STAGE2A_CONFIG, dataset_type)
data_combiner = DataCombiner(STAGE2A_CONFIG, dataset_type)
binning_processor = BinningProcessor(STAGE2A_CONFIG, dataset_type)
regression_preparer = RegressionDataPreparer(STAGE2A_CONFIG, dataset_type)
feature_preprocessor = FeaturePreprocessor(STAGE2A_CONFIG, dataset_type)
data_saver = DataSaver(STAGE2A_CONFIG, dataset_type)
feature_mapper = FeatureMapper(STAGE2A_CONFIG, dataset_type)

if STAGE2A_CONFIG['seq2seq']['enabled']:
    seq2seq_preparer = Seq2SeqDataPreparer(STAGE2A_CONFIG, dataset_type)
    print("Seq2seq data preparer initialized")

print("Components initialized")
print("=" * 60)

Initializing processing components...
Components initialized


In [14]:
# Step 3: Process spectral data
print("Processing spectral data...")

# For regression, we need binned spectra
print("Processing binned spectra for regression...")
spectra_sequences_binned = spectral_processor.process(raw_data, use_binning=True)

# For seq2seq, we need full resolution spectra
if STAGE2A_CONFIG['seq2seq']['enabled']:
    print("\nProcessing full resolution spectra for seq2seq...")
    spectra_sequences_full = spectral_processor.process(raw_data, use_binning=False)

# Free up raw data after processing
del raw_data

print(f"\nMemory usage after spectral processing: {Utilities.get_memory_usage()}")
print("=" * 60)

2025-06-23 09:05:17,749 - INFO - Processing spectral sequences (binning=enabled)...


Processing spectral data...
Processing binned spectra for regression...


Processing spectra: 100%|██████████| 2720/2720 [00:07<00:00, 386.07it/s] 
2025-06-23 09:05:25,887 - INFO - Maximum observed m/z across all spectra: 683.0
2025-06-23 09:05:25,888 - INFO - Configured max_mz for binning: 499
2025-06-23 09:05:25,889 - INFO - 10 spectra (0.37%) have peaks beyond configured max_mz
2025-06-23 09:05:25,891 - INFO - Processed spectral sequences for 2720 compounds



Memory usage after spectral processing: 646.0 MB (RSS), 0.5% of total


In [15]:
# Step 4: Combine data
print("Combining molecular features with spectral data...")

# Combine for regression (using binned spectra)
combined_data_binned = data_combiner.process(spectra_sequences_binned, molecular_features)

# Also combine for seq2seq if enabled (using full resolution)
if STAGE2A_CONFIG['seq2seq']['enabled']:
    combined_data_full = data_combiner.process(spectra_sequences_full, molecular_features)
    del spectra_sequences_full  # Free memory

# Free up memory after combining
del spectra_sequences_binned
del molecular_features

print(f"\nMemory usage after data combination: {Utilities.get_memory_usage()}")
print("=" * 60)

Combining molecular features with spectral data...


Combining data: 100%|██████████| 2720/2720 [00:00<00:00, 82479.68it/s]
2025-06-23 09:05:25,948 - INFO - Combined data: 2720 compounds, missing features for 0



Memory usage after data combination: 647.7 MB (RSS), 0.5% of total


In [16]:
# Step 5: Bin spectral data for regression
print("Creating binned representations for regression...")
binned_data = binning_processor.process(combined_data_binned)

# Free up the combined data after binning
del combined_data_binned

print(f"\nMemory usage after binning: {Utilities.get_memory_usage()}")
print("=" * 60)

2025-06-23 09:05:25,966 - INFO - Maximum observed m/z for binning: 683.0


Creating binned representations for regression...


Binning spectra: 100%|██████████| 2720/2720 [00:08<00:00, 316.04it/s] 
2025-06-23 09:05:34,992 - INFO - Created binned representations for 2720 compounds
2025-06-23 09:05:34,993 - INFO - Memory usage after binning: 944.8 MB (RSS), 0.7% of total



Memory usage after binning: 944.8 MB (RSS), 0.7% of total


In [17]:
# Step 6: Save visualization sample for Stage 2B
if STAGE2A_CONFIG['processing']['save_visualization_sample']:
    print("Saving visualization sample...")
    visualization_sample = {k: binned_data[k] for k in random.sample(list(binned_data.keys()), 
                                                                     min(STAGE2A_CONFIG['processing']['visualization_sample_size'], 
                                                                         len(binned_data)))}
    
    # Save visualization data
    viz_data_path = os.path.join(STAGE2A_CONFIG['paths']['output_dir'](dataset_type), 'visualization_data.pkl')
    dump(visualization_sample, viz_data_path)
    print(f"Saved {len(visualization_sample)} samples for visualization to {viz_data_path}")
    
    # Also save spectral processor stats
    viz_stats = {
        'max_observed_mz': spectral_processor.max_observed_mz,
        'binning_processor_max_mz': binning_processor.max_observed_mz,
        'spectral_config': STAGE2A_CONFIG['spectral']
    }
    viz_stats_path = os.path.join(STAGE2A_CONFIG['paths']['output_dir'](dataset_type), 'visualization_stats.json')
    with open(viz_stats_path, 'w') as f:
        json.dump(viz_stats, f, indent=2)
    
print("=" * 60)

Saving visualization sample...
Saved 100 samples for visualization to ../data/results/hpj/full_featurised/visualization_data.pkl


In [18]:
# Step 7: Prepare regression data
print("Preparing regression data...")
reg_result = regression_preparer.process(binned_data)

if reg_result is None:
    print("ERROR: Regression data preparation failed")
else:
    X, y, all_smiles, feature_metadata = reg_result
    print(f"\nRegression data prepared:")
    print(f"  Feature matrix shape: {X.shape}")
    print(f"  Target matrix shape: {y.shape}")
    print(f"  Number of samples: {len(all_smiles)}")
    
    # Free up binned data after regression preparation (keep if needed for seq2seq)
    if not STAGE2A_CONFIG['seq2seq']['enabled']:
        del binned_data
    
    print(f"\nMemory usage after regression preparation: {Utilities.get_memory_usage()}")
    print("=" * 60)

2025-06-23 09:05:36,111 - INFO - Extracting features and targets for regression...


Preparing regression data...


Preparing regression data: 100%|██████████| 2720/2720 [00:01<00:00, 1799.58it/s]
2025-06-23 09:05:37,669 - INFO - Memory usage after regression data preparation: 1240.9 MB (RSS), 0.9% of total
2025-06-23 09:05:37,670 - WARNING - Feature dimension mismatch: expected 7181, got 7377
2025-06-23 09:05:37,671 - INFO - Prepared regression matrices - X: (2720, 7377), y: (2720, 500)



Regression data prepared:
  Feature matrix shape: (2720, 7377)
  Target matrix shape: (2720, 500)
  Number of samples: 2720

Memory usage after regression preparation: 1240.9 MB (RSS), 0.9% of total


In [19]:
# Step 8: Preprocess features
print("Preprocessing features...")
X_processed = feature_preprocessor.process(X, fit=True, metadata=feature_metadata)

# Save preprocessor
output_dir = STAGE2A_CONFIG['paths']['output_dir'](dataset_type)
preprocessor_output = os.path.join(output_dir, 'feature_preprocessor.pkl')
feature_preprocessor.save(preprocessor_output)
print(f"Feature preprocessor saved to {preprocessor_output}")

print(f"\nPreprocessed feature shape: {X_processed.shape}")
print(f"Memory usage after feature preprocessing: {Utilities.get_memory_usage()}")
print("=" * 60)

2025-06-23 09:05:37,841 - INFO - Found 8 NaN values and 0 infinity values across 4 features


Preprocessing features...


2025-06-23 09:05:39,107 - INFO - Detected 7206 binary features
2025-06-23 09:05:39,174 - INFO - Dropped 4 features with invalid values (4 with NaN, 0 with infinity)
2025-06-23 09:05:39,712 - INFO - Dropped 236 features with variance below 1e-08
2025-06-23 09:05:39,731 - INFO - Final feature count: 7137 of 7377 original features
2025-06-23 09:05:40,114 - INFO - Saved feature preprocessor to ../data/results/hpj/full_featurised/feature_preprocessor.pkl


Feature preprocessor saved to ../data/results/hpj/full_featurised/feature_preprocessor.pkl

Preprocessed feature shape: (2720, 7137)
Memory usage after feature preprocessing: 1812.6 MB (RSS), 1.4% of total


In [20]:
# Step 9: Split data
print("Splitting data into train/val/test sets...")
print(f"Split ratios: train={STAGE2A_CONFIG['data_split']['train_size']}, "
      f"val={STAGE2A_CONFIG['data_split']['val_size']}, "
      f"test={STAGE2A_CONFIG['data_split']['test_size']}")

# Set random seed for reproducibility
random.seed(STAGE2A_CONFIG['data_split']['random_seed'])
np.random.seed(STAGE2A_CONFIG['data_split']['random_seed'])

# First split into train and temp
train_size_ratio = STAGE2A_CONFIG['data_split']['train_size']
X_train, X_temp, y_train, y_temp, smiles_train, smiles_temp = train_test_split(
    X_processed, y, all_smiles, 
    test_size=1-train_size_ratio, 
    random_state=STAGE2A_CONFIG['data_split']['random_seed']
)

# Then split temp into validation and test
val_ratio = STAGE2A_CONFIG['data_split']['val_size'] / (STAGE2A_CONFIG['data_split']['val_size'] + STAGE2A_CONFIG['data_split']['test_size'])
X_val, X_test, y_val, y_test, smiles_val, smiles_test = train_test_split(
    X_temp, y_temp, smiles_temp,
    test_size=1-val_ratio,
    random_state=STAGE2A_CONFIG['data_split']['random_seed']
)

# Free temporary variables
del X_temp, y_temp, smiles_temp

print(f"\nData split complete:")
print(f"  Train: {len(X_train)} samples")
print(f"  Val: {len(X_val)} samples")
print(f"  Test: {len(X_test)} samples")
print("=" * 60)

Splitting data into train/val/test sets...
Split ratios: train=0.8, val=0.1, test=0.1

Data split complete:
  Train: 2176 samples
  Val: 272 samples
  Test: 272 samples


In [21]:
# Step 10: Scale targets if enabled
print("Processing target scaling...")
y_train_scaled, y_val_scaled, y_test_scaled = regression_preparer.scale_targets(y_train, y_val, y_test)

# Save target scaler if used
if STAGE2A_CONFIG['target_scaling']['enabled'] and regression_preparer.target_scaler is not None:
    target_scaler_path = os.path.join(output_dir, 'target_scaler.pkl')
    data_saver.save_target_scaler(regression_preparer.target_scaler, target_scaler_path)
    print(f"Target scaler saved to {target_scaler_path}")

# Determine if targets were scaled
targets_scaled = STAGE2A_CONFIG['target_scaling']['enabled'] and regression_preparer.target_scaler is not None
print(f"Targets scaled: {targets_scaled}")
print("=" * 60)

2025-06-23 09:05:40,519 - INFO - Target scaling disabled, returning original targets


Processing target scaling...
Targets scaled: False


In [22]:
# Step 11: Update and save feature metadata
print("Updating feature metadata...")
updated_metadata = feature_preprocessor.update_metadata(feature_metadata)
feature_mapper.collect_feature_info(updated_metadata, regression_preparer.target_scaler)
feature_mapper.save_mapping()
print("Feature mapping saved")
print("=" * 60)

2025-06-23 09:05:40,568 - INFO - Updated metadata: 7137 valid features retained out of 7377.


Updating feature metadata...


2025-06-23 09:05:40,944 - INFO - Saved feature mapping for 7137 features to ../data/results/hpj/full_featurised/feature_mapping.jsonl
2025-06-23 09:05:40,947 - INFO - Saved complete feature info to ../data/results/hpj/full_featurised/feature_mapping_full.json


Feature mapping saved


In [23]:
# Step 12: Save regression splits
print("Saving regression data splits...")

# Define output paths
train_output = os.path.join(output_dir, 'train_data.jsonl')
val_output = os.path.join(output_dir, 'val_data.jsonl')
test_output = os.path.join(output_dir, 'test_data.jsonl')

train_indices = list(range(len(smiles_train)))
val_indices = list(range(len(smiles_val)))
test_indices = list(range(len(smiles_test)))

# Save with scaling information
data_saver.save_split(smiles_train, X_train, y_train_scaled, train_indices, train_output, 
                     is_features_scaled=True, is_targets_scaled=targets_scaled)
data_saver.save_split(smiles_val, X_val, y_val_scaled, val_indices, val_output,
                     is_features_scaled=True, is_targets_scaled=targets_scaled)
data_saver.save_split(smiles_test, X_test, y_test_scaled, test_indices, test_output,
                     is_features_scaled=True, is_targets_scaled=targets_scaled)

print("\nRegression data saved to:")
print(f"  {train_output}")
print(f"  {val_output}")
print(f"  {test_output}")

# Save dataset configuration
config_path = os.path.join(output_dir, 'dataset_config.json')
with open(config_path, 'w') as f:
    json.dump({
        'dataset_type': dataset_type,
        'stage2a_complete': True,
        'num_samples': len(all_smiles),
        'feature_dim': X_processed.shape[1],
        'target_dim': y.shape[1],
        'spectral_config': STAGE2A_CONFIG['spectral'],
        'feature_scaling_config': STAGE2A_CONFIG['feature_scaling'],
        'target_scaling_config': STAGE2A_CONFIG['target_scaling'],
        'features_scaled': True,
        'targets_scaled': targets_scaled,
        'visualization_saved': STAGE2A_CONFIG['processing']['save_visualization_sample']
    }, f, indent=2)
print(f"\nSaved configuration to {config_path}")
print("=" * 60)

Saving regression data splits...


Saving to train_data.jsonl: 100%|██████████| 2176/2176 [00:06<00:00, 328.68it/s]
2025-06-23 09:05:47,849 - INFO - Saved 2176 samples to ../data/results/hpj/full_featurised/train_data.jsonl
Saving to val_data.jsonl: 100%|██████████| 272/272 [00:00<00:00, 454.30it/s]
2025-06-23 09:05:48,472 - INFO - Saved 272 samples to ../data/results/hpj/full_featurised/val_data.jsonl
Saving to test_data.jsonl: 100%|██████████| 272/272 [00:00<00:00, 441.84it/s]
2025-06-23 09:05:49,097 - INFO - Saved 272 samples to ../data/results/hpj/full_featurised/test_data.jsonl



Regression data saved to:
  ../data/results/hpj/full_featurised/train_data.jsonl
  ../data/results/hpj/full_featurised/val_data.jsonl
  ../data/results/hpj/full_featurised/test_data.jsonl

Saved configuration to ../data/results/hpj/full_featurised/dataset_config.json


In [24]:
# Step 13: Generate seq2seq data if enabled
if STAGE2A_CONFIG['seq2seq']['enabled']:
    print("GENERATING SEQ2SEQ DATASET")
    print("=" * 60)
    
    # Process seq2seq data
    seq2seq_data = seq2seq_preparer.process(X_processed, combined_data_full, all_smiles)
    
    # Split seq2seq data using same indices as regression
    # Create mapping from SMILES to indices
    smiles_to_idx = {smiles: idx for idx, smiles in enumerate(all_smiles)}
    
    # Get indices for each split
    train_indices_seq2seq = [smiles_to_idx[smiles] for smiles in smiles_train]
    val_indices_seq2seq = [smiles_to_idx[smiles] for smiles in smiles_val]
    test_indices_seq2seq = [smiles_to_idx[smiles] for smiles in smiles_test]
    
    # Create seq2seq splits
    seq2seq_train = [seq2seq_data[i] for i in train_indices_seq2seq]
    seq2seq_val = [seq2seq_data[i] for i in val_indices_seq2seq]
    seq2seq_test = [seq2seq_data[i] for i in test_indices_seq2seq]
    
    # Define seq2seq output paths
    seq2seq_output_dir = os.path.join(
        STAGE2A_CONFIG['paths']['results_dir'](dataset_type),
        STAGE2A_CONFIG['seq2seq']['output_subdir']
    )
    os.makedirs(seq2seq_output_dir, exist_ok=True)
    
    print(f"\nSaving seq2seq data to {seq2seq_output_dir}...")
    
    # Save seq2seq splits
    data_saver.save_seq2seq_split(
        seq2seq_train, 
        list(range(len(seq2seq_train))), 
        os.path.join(seq2seq_output_dir, 'train_data.jsonl')
    )
    data_saver.save_seq2seq_split(
        seq2seq_val,
        list(range(len(seq2seq_val))),
        os.path.join(seq2seq_output_dir, 'val_data.jsonl')
    )
    data_saver.save_seq2seq_split(
        seq2seq_test,
        list(range(len(seq2seq_test))),
        os.path.join(seq2seq_output_dir, 'test_data.jsonl')
    )
    
    # Save PCA transformer
    pca_path = os.path.join(seq2seq_output_dir, 'pca_transformer.pkl')
    seq2seq_preparer.save_pca(pca_path)
    
    # Save seq2seq config
    seq2seq_config = {
        'pca_components': STAGE2A_CONFIG['seq2seq']['pca_components'],
        'max_sequence_length': STAGE2A_CONFIG['seq2seq']['max_sequence_length'],
        'input_dim': STAGE2A_CONFIG['seq2seq']['pca_components'],
        'output_dim': 2,  # (mz, intensity)
        'dataset_type': dataset_type,
        'pca_explained_variance': float(np.sum(seq2seq_preparer.pca.explained_variance_ratio_))
    }
    
    seq2seq_config_path = os.path.join(seq2seq_output_dir, 'seq2seq_config.json')
    with open(seq2seq_config_path, 'w') as f:
        json.dump(seq2seq_config, f, indent=2)
    
    print(f"\nSeq2seq data saved to {seq2seq_output_dir}")
    print(f"PCA explained variance: {seq2seq_config['pca_explained_variance']:.2%}")
    
    # Clean up
    del combined_data_full
    del seq2seq_data
    del seq2seq_train
    del seq2seq_val
    del seq2seq_test

# Free up memory after saving data splits
del X, y, X_processed
del X_train, y_train, smiles_train, y_train_scaled
del X_val, y_val, smiles_val, y_val_scaled
del X_test, y_test, smiles_test, y_test_scaled

print(f"\nMemory usage after saving all data: {Utilities.get_memory_usage()}")
print("=" * 60)


Memory usage after saving all data: 1887.7 MB (RSS), 1.4% of total


## 13. Pipeline Summary

Display final summary of Stage 2A processing.

In [25]:
# Log summary
print("\n" + "="*60)
print("STAGE 2A SUMMARY")
print("="*60)

feature_dimensions = ""
if feature_preprocessor.valid_feature_mask is not None:
    feature_dimensions = f"\n  Feature Dimensions: {np.sum(feature_preprocessor.valid_feature_mask)}"
    
# Include statistics about m/z range coverage in summary
max_observed_mz_info = ""
if hasattr(spectral_processor, 'max_observed_mz') and spectral_processor.max_observed_mz > 0:
    max_observed_mz_info = (f"\n  Max Observed m/z: {spectral_processor.max_observed_mz:.1f} " +
                           f"(configured max_mz: {STAGE2A_CONFIG['spectral']['max_mz']})")
    
print(f"  Dataset Type: {dataset_type}")
print(f"  Total Samples: {len(all_smiles)}")
print(feature_dimensions)
print(max_observed_mz_info)

print(f"\n  Scaling Configuration:")
print(f"    Features:")
print(f"      - Continuous: {STAGE2A_CONFIG['feature_scaling']['scale_continuous']} (method: {STAGE2A_CONFIG['feature_scaling']['continuous_scaling_method']})")
print(f"      - Log transform: {STAGE2A_CONFIG['feature_scaling']['apply_log_transform']}")
print(f"      - Binary: {STAGE2A_CONFIG['feature_scaling']['scale_binary']} (method: {STAGE2A_CONFIG['feature_scaling']['binary_scaling_method']})")
print(f"    Targets:")
print(f"      - Enabled: {STAGE2A_CONFIG['target_scaling']['enabled']} (method: {STAGE2A_CONFIG['target_scaling']['scaling_method']})")
print(f"      - Applied: {targets_scaled}")

print(f"\n  Regression Output Files:")
print(f"    - {train_output}")
print(f"    - {val_output}")
print(f"    - {test_output}")
print(f"    - {preprocessor_output}")
print(f"    - {feature_mapper.output_file}")
if targets_scaled:
    print(f"    - {target_scaler_path}")

if STAGE2A_CONFIG['processing']['save_visualization_sample']:
    print(f"\n  Visualization Files:")
    print(f"    - {viz_data_path}")
    print(f"    - {viz_stats_path}")

if STAGE2A_CONFIG['seq2seq']['enabled']:
    print(f"\n  Seq2Seq Output Files:")
    print(f"    - {os.path.join(seq2seq_output_dir, 'train_data.jsonl')}")
    print(f"    - {os.path.join(seq2seq_output_dir, 'val_data.jsonl')}")
    print(f"    - {os.path.join(seq2seq_output_dir, 'test_data.jsonl')}")
    print(f"    - {os.path.join(seq2seq_output_dir, 'pca_transformer.pkl')}")
    print(f"    - {os.path.join(seq2seq_output_dir, 'seq2seq_config.json')}")

print(f"\n  Final Memory Usage: {Utilities.get_memory_usage()}")
print("\n" + "="*60)
print("STAGE 2A COMPLETE!")
print("Next: Run 02b_featurization_visualization.ipynb for visualizations")
print("="*60)


STAGE 2A SUMMARY
  Dataset Type: hpj
  Total Samples: 2720

  Feature Dimensions: 7137

  Max Observed m/z: 683.0 (configured max_mz: 499)

  Scaling Configuration:
    Features:
      - Continuous: False (method: standard)
      - Log transform: False
      - Binary: False (method: none)
    Targets:
      - Enabled: False (method: minmax)
      - Applied: False

  Regression Output Files:
    - ../data/results/hpj/full_featurised/train_data.jsonl
    - ../data/results/hpj/full_featurised/val_data.jsonl
    - ../data/results/hpj/full_featurised/test_data.jsonl
    - ../data/results/hpj/full_featurised/feature_preprocessor.pkl
    - ../data/results/hpj/full_featurised/feature_mapping.jsonl

  Visualization Files:
    - ../data/results/hpj/full_featurised/visualization_data.pkl
    - ../data/results/hpj/full_featurised/visualization_stats.json

  Final Memory Usage: 1887.7 MB (RSS), 1.4% of total

STAGE 2A COMPLETE!
Next: Run 02b_featurization_visualization.ipynb for visualizations


## 14. Output Summary and Next Steps

### Files Generated:

**Regression Format (data/results/{dataset}/full_featurised/):**
- `train_data.jsonl` - Training set with scaled features and targets
- `val_data.jsonl` - Validation set
- `test_data.jsonl` - Test set
- `feature_preprocessor.pkl` - Feature scaling and preprocessing state
- `target_scaler.pkl` - Target scaling state (if enabled)
- `feature_mapping.jsonl` - Feature metadata for interpretability
- `feature_mapping_full.json` - Complete feature and scaling information
- `dataset_config.json` - Configuration metadata
- `visualization_data.pkl` - Sample data for Stage 2B
- `visualization_stats.json` - Statistics for Stage 2B

**Seq2Seq Format (data/results/{dataset}/seq2seq_featurised/):**
- `train_data.jsonl` - Training set with PCA features and full resolution spectra
- `val_data.jsonl` - Validation set
- `test_data.jsonl` - Test set
- `pca_transformer.pkl` - PCA transformation matrix
- `seq2seq_config.json` - Configuration and metadata

### Next Steps:
Run Stage 2B (02b_featurization_visualization.ipynb) to create comprehensive visualizations of the processed data.


In [ ]:
# Display file sizes
import os

print("Generated file sizes:")
print("\nRegression files:")
output_dir = STAGE2A_CONFIG['paths']['output_dir'](dataset_type)
if os.path.exists(output_dir):
    for file in sorted(os.listdir(output_dir)):
        file_path = os.path.join(output_dir, file)
        if os.path.isfile(file_path):
            size_mb = os.path.getsize(file_path) / (1024 * 1024)
            print(f"  - {file}: {size_mb:.2f} MB")

if STAGE2A_CONFIG['seq2seq']['enabled']:
    print("\nSeq2seq files:")
    seq2seq_dir = os.path.join(STAGE2A_CONFIG['paths']['results_dir'](dataset_type), 
                               STAGE2A_CONFIG['seq2seq']['output_subdir'])
    if os.path.exists(seq2seq_dir):
        for file in sorted(os.listdir(seq2seq_dir)):
            file_path = os.path.join(seq2seq_dir, file)
            if os.path.isfile(file_path):
                size_mb = os.path.getsize(file_path) / (1024 * 1024)
                print(f"  - {file}: {size_mb:.2f} MB")

print("\nTemporary files (can be deleted after pipeline completion):")
temp_dir = STAGE2A_CONFIG['paths']['temp_dir'](dataset_type)
if os.path.exists(temp_dir):
    total_temp_size = 0
    for file in os.listdir(temp_dir):
        file_path = os.path.join(temp_dir, file)
        if os.path.isfile(file_path):
            size_mb = os.path.getsize(file_path) / (1024 * 1024)
            total_temp_size += size_mb
            print(f"  - {file}: {size_mb:.2f} MB")
    print(f"\nTotal temporary storage: {total_temp_size:.2f} MB")

Generated file sizes:

Regression files:
  - dataset_config.json: 0.00 MB
  - feature_mapping.jsonl: 0.56 MB
  - feature_mapping_full.json: 0.61 MB
  - feature_preprocessor.pkl: 0.02 MB
  - target_scaler.pkl: 0.02 MB
  - test_data.jsonl: 10.78 MB
  - train_data.jsonl: 86.10 MB
  - val_data.jsonl: 10.77 MB
  - visualization_data.pkl: 3.63 MB
  - visualization_stats.json: 0.00 MB

Temporary files (can be deleted after pipeline completion):
  - molecular_features_3d.jsonl: 2.21 MB
  - dataset_config.json: 0.00 MB
  - dataset_metadata_3d.json: 0.00 MB
  - 3d_features.pkl: 10.96 MB
  - corrupted_records.jsonl: 0.00 MB
  - corrupted_3d_records.jsonl: 0.00 MB
  - dataset_config_3d.json: 0.00 MB
  - molecular_features.jsonl: 111.92 MB
  - feature_importance_3d.json: 0.00 MB
  - raw_spectral_data.jsonl: 16.40 MB
  - raw_spectral_data_3d.jsonl: 7.40 MB

Total temporary storage: 148.89 MB


: 